In [3]:
import pandas as pd
import numpy as np
import re
import string
import math
import hashlib
import os
import pickle
import json
import joblib

from sklearn.feature_selection import mutual_info_classif
from sklearn import preprocessing
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.metrics import confusion_matrix,roc_auc_score,roc_curve,classification_report,auc,precision_score,recall_score,precision_recall_curve,median_absolute_error
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import lightgbm as lgb


### Data loading

In [4]:
with open(os.path.join("data", "train.csv")) as f:
    # read the training dataset
    df = pd.read_csv(f, index_col = 'id')

with open(os.path.join("data", "test.csv")) as f:
    # read the test dataset
    X_test = pd.read_csv(f, index_col = 'id')


X = df.drop(['Hardness'], axis=1)
y = df['Hardness']

### Baseline Model

In [5]:
with open(os.path.join('pickles/baseline', 'columns.json')) as fh:
    columns = json.load(fh)

with open(os.path.join('pickles/baseline', 'dtypes.pickle'), 'rb') as fh:
    dtypes = pickle.load(fh)

with open(os.path.join('pickles/baseline', 'pipeline.pickle'), 'rb') as fh:
    pipeline_base = joblib.load(fh)

features = X.columns

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.3, random_state = 42)

X_train_baseline = X_train[features]

pipeline_base.fit(X_train_baseline, y_train)

Pipeline(steps=[('processing', MinMaxScaler()),
                ('regressor', LinearRegression())])

In [2]:
y_val_pred = pipeline_base.predict(X_val)

print('Median Absolute Error score of {}: {}'.format('baseline',median_absolute_error(y_val, y_val_pred)))

NameError: name 'pipeline_base' is not defined

# Model improvement

In [27]:
process = preprocessing.MinMaxScaler()

pipeline_light = Pipeline(steps=[('processing',process),
                     ('regressor', lgb.LGBMRegressor(learning_rate=0.09,max_depth=-15,random_state=42))])

pipeline_rf = Pipeline(steps = [('processing', process),
                                ('regressor',RandomForestRegressor(max_depth=15,random_state = 42))])

pipeline_dt = Pipeline(steps=[('processing', process),
                              ('regressor', DecisionTreeRegressor(max_depth=15,random_state = 42))])

pipeline_svr = Pipeline(steps=[('processing', process),
                              ('regressor', SVR())])


pipelines = [pipeline_base, pipeline_light, pipeline_rf, pipeline_dt, pipeline_svr]
pipe_dict = {0: 'Baseline', 1: 'LightGBM', 2:'Random Forest', 3: 'Decision Tree', 4: 'SVR'}

for pipe in pipelines:
    pipe.fit(X_train, y_train)

best_precision = 0.0
best_regressor = 0
best_medae = 10
best_pipeline = ''
prediction_dict={}

for i, model in enumerate(pipelines) : 
    y_val_pred = model.predict(X_val)
    medae = median_absolute_error(y_val, y_val_pred)

    prediction_dict[pipe_dict[i]] = medae

    if medae < best_medae:
        best_medae = medae
        best_pipeline = model
        best_regressor = i
print('Best Median Absolute Error with {} | MedAE: {}'.format(pipe_dict[best_regressor],best_medae))

prediction_dict

Best Median Absolute Error with LightGBM | MedAE: 0.6452822745961022


{'Baseline': 0.9555663913173742,
 'LightGBM': 0.6452822745961022,
 'Random Forest': 0.645478039694277,
 'Decision Tree': 0.7000000000000002,
 'SVR': 0.6527384090608557}

### LightGBM optimization

In [6]:
# light_grid = lgb.LGBMRegressor(random_state=42)


# parameters = {
#     # 'task' : ['predict'],
#     'boosting': ['gbdt' ],
#     'objective': ['mae'],
#     #'num_iterations': [  1500, 2000  ],
#     'learning_rate':[  0.05, 0.005 ],
#     'num_leaves':[ 7, 15 ],
#     'max_depth' :[ 10,15],
#     #'min_data_in_leaf':[15,25],
#     'feature_fraction': [ 0.6, 0.8],
#     'bagging_fraction': [  0.6, 0.8 ],
#     #'bagging_freq': [   100, 200, 400  ], 
# }

# grid_lgb = GridSearchCV(light_grid, param_grid = parameters, n_jobs=6, verbose=3, scoring = 'neg_median_absolute_error', cv = 5)
# scaler = preprocessing.MinMaxScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# grid_lgb.fit(X_train_scaled,y_train)
 

# print('best params')
# print(grid_lgb.best_params_)

Fitting 5 folds for each of 32 candidates, totalling 160 fits


[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
[LightGBM] [Warning] bagging_fraction is set=0.6, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6
[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
[LightGBM] [Warning] bagging_fraction is set=0.6, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
[LightGBM] [Warning] bagging_fraction is set=0.6, subsample=1.0 will b

In [7]:
# X_val_scaled = scaler.transform(X_val)
# y_val_pred_grid = grid_lgb.predict(X_val_scaled)
# medae_grid = median_absolute_error(y_val, y_val_pred_grid)
# print(medae_grid)
# print(grid_lgb.best_params_)

0.5382103325544856


In [20]:
# Pipeline using the grid-optimized lightGBM model
scaler = preprocessing.MinMaxScaler()
pipeline_grid_light = Pipeline([('scaler', scaler),
                          ('regressor', lgb.LGBMRegressor(random_state=42, bagging_fraction = 0.6,
                                                          boosting = 'gbdt', feature_fraction = 0.6,
                                                          learning_rate = 0.05, max_depth = 15,
                                                          num_leaves = 15, objective= 'mae'))])

pipeline_grid_light.fit(X, y)

y_test_pred = pipeline_grid_light.predict(X_test)

# output = pd.DataFrame({'id': X_test.index,
#                        'emission': y_test_pred})

# from datetime import date, datetime
# now = datetime.now()
# today_str = now.strftime("%d%m%Y_%H%M")

# subm_file_name = "submissions/submission_"+today_str+".csv"

# output.to_csv(subm_file_name, index=False)



[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
[LightGBM] [Warning] bagging_fraction is set=0.6, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6


In [10]:
scores = cross_val_score(pipeline_grid_light, X, y, cv=5, scoring='neg_median_absolute_error')
mean_error = round(np.mean(scores), 2)
var_error = round(np.var(scores), 2)
print(f'---\nValidation error:\nMean: {mean_error}% | Variance: {var_error}')
scores

[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
[LightGBM] [Warning] bagging_fraction is set=0.6, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
[LightGBM] [Warning] bagging_fraction is set=0.6, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
[LightGBM] [Warning] bagging_fraction is set=0.6, subsample=1.0 will b

array([-0.51842542, -0.53373033, -0.60288837, -0.5647964 , -0.59549626])

Score on Kaggle: 0.5141

### SVR optimization

In [11]:
# SVR_grid = SVR()


# parameters = {
#     'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
#     'degree': [3,6,9],
#     'gamma': ['scale','auto']

# }

# grid_svr = GridSearchCV(SVR_grid, param_grid = parameters, n_jobs=6, verbose=3, scoring = 'neg_median_absolute_error', cv = 5)
# scaler = preprocessing.MinMaxScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# grid_svr.fit(X_train_scaled,y_train)
 

# print('best params')
# print(grid_svr.best_params_)

Fitting 5 folds for each of 24 candidates, totalling 120 fits


[CV 4/5] END degree=3, gamma=scale, kernel=linear;, score=-0.795 total time=   3.4s
[CV 3/5] END degree=3, gamma=scale, kernel=linear;, score=-0.822 total time=   3.4s
[CV 5/5] END degree=3, gamma=scale, kernel=linear;, score=-0.833 total time=   3.4s
[CV 1/5] END degree=3, gamma=scale, kernel=linear;, score=-0.826 total time=   3.4s
[CV 2/5] END degree=3, gamma=scale, kernel=linear;, score=-0.837 total time=   3.5s
[CV 1/5] END degree=3, gamma=scale, kernel=poly;, score=-0.733 total time=   4.6s
[CV 4/5] END degree=3, gamma=scale, kernel=poly;, score=-0.702 total time=   3.5s
[CV 5/5] END degree=3, gamma=scale, kernel=poly;, score=-0.737 total time=   3.6s
[CV 2/5] END degree=3, gamma=scale, kernel=poly;, score=-0.785 total time=   3.7s
[CV 3/5] END degree=3, gamma=scale, kernel=poly;, score=-0.782 total time=   3.9s
[CV 1/5] END degree=3, gamma=scale, kernel=rbf;, score=-0.699 total time=   4.5s
[CV 2/5] END degree=3, gamma=scale, kernel=rbf;, score=-0.722 total time=   4.5s
[CV 3/5]

In [12]:
# X_val_scaled = scaler.transform(X_val)
# y_val_pred_grid = grid_svr.predict(X_val_scaled)
# medae_grid = median_absolute_error(y_val, y_val_pred_grid)
# print(medae_grid)

0.6628639047729248


In [19]:
# Pipeline using the grid-optimized SVR model
scaler = preprocessing.MinMaxScaler()

pipeline_grid_svr = Pipeline([('scaler', scaler),
                          ('regressor', SVR(degree = 6, gamma = 'scale', kernel = 'poly'))])

pipeline_grid_svr.fit(X, y)

y_test_pred = pipeline_grid_svr.predict(X_test)

output = pd.DataFrame({'id': X_test.index,
                       'emission': y_test_pred})

from datetime import date, datetime
now = datetime.now()
today_str = now.strftime("%d%m%Y_%H%M")

subm_file_name = "submissions/submission_"+today_str+".csv"

output.to_csv(subm_file_name, index=False)


Score on Kaggle: 0.6566

#### Preparing submission file
New model (LightGBM) with all features included.

In [13]:
# pipeline_light.fit(X,y)

# y_test_pred = pipeline_light.predict(X_test)

# output = pd.DataFrame({'id': X_test.index,
#                        'emission': y_test_pred})

# from datetime import date, datetime
# now = datetime.now()
# today_str = now.strftime("%d%m%Y_%H%M")

# subm_file_name = "submissions/submission_"+today_str+".csv"

# output.to_csv(subm_file_name, index=False)

### Feature Engineering 

In [14]:
# Using only the top 5 correlation features
best_corr_feats = ['atomicweight_Average', 'density_Average',# 'allelectrons_Average',
                   'el_neg_chi_Average', 'ionenergy_Average', 'R_cov_element_Average']

X_corr = X[best_corr_feats]

X_train_corr, X_val_corr, y_train_corr, y_val_corr = train_test_split(X_corr, y, test_size = 0.3, random_state = 42)

In [24]:
process = preprocessing.MinMaxScaler()

pipeline_light_2 = Pipeline(steps=[('processing',process),
                     ('regressor', lgb.LGBMRegressor(random_state=42, bagging_fraction = 0.6,
                                                          boosting = 'gbdt', feature_fraction = 0.6,
                                                          learning_rate = 0.05, max_depth = 15,
                                                          num_leaves = 15, objective= 'mae'))])

pipeline_rf_2 = Pipeline(steps = [('processing', process),
                                ('regressor',RandomForestRegressor(max_depth=15,random_state = 42))])

pipeline_dt_2 = Pipeline(steps=[('processing', process),
                              ('regressor', DecisionTreeRegressor(max_depth=15,random_state = 42))])

pipeline_svr_2 = Pipeline(steps=[('processing', process),
                              ('regressor', SVR(degree = 6, gamma = 'scale', kernel = 'poly'))])


pipelines_2 = [pipeline_base, pipeline_light_2, pipeline_rf_2, pipeline_dt_2, pipeline_svr_2]
pipe_dict_2 = {0: 'Baseline', 1: 'LightGBM', 2:'Random Forest', 3: 'Decision Tree', 4: 'SVR'}

for pipe in pipelines_2:
    pipe.fit(X_train_corr, y_train_corr)

best_precision = 0.0
best_regressor = 0
best_medae = 10
best_pipeline = ''
prediction_dict_2={}

for i, model in enumerate(pipelines_2) : 
    y_val_pred = model.predict(X_val_corr)
    medae = median_absolute_error(y_val_corr, y_val_pred)

    prediction_dict_2[pipe_dict_2[i]] = medae

    if medae < best_medae:
        best_medae = medae
        best_pipeline = model
        best_regressor = i
print('Best Median Absolute Error with {} | MedAE: {}'.format(pipe_dict_2[best_regressor],best_medae))

prediction_dict_2

[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
[LightGBM] [Warning] bagging_fraction is set=0.6, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6
Best Median Absolute Error with LightGBM | MedAE: 0.5845266262717335


{'Baseline': 0.9935564263469043,
 'LightGBM': 0.5845266262717335,
 'Random Forest': 0.6731373350042418,
 'Decision Tree': 0.6348837209302376,
 'SVR': 0.7117191991904059}

In [44]:
# Checking which feature has the biggest impact on score
medae_dict = {}
for i in X.columns:
    X_i =  X.copy()
    X_i = X_i.drop(columns=i)
    X_train_i, X_val_i, y_train_i, y_val_i = train_test_split(X_i, y, test_size=0.3, random_state=42)
    pipeline_light_2.fit(X_train_i, y_train_i)
    y_val_pred_i = pipeline_light_2.predict(X_val_i)
    medae_i = median_absolute_error(y_val_i, y_val_pred_i)
    medae_dict[i] = medae_i

[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
[LightGBM] [Warning] bagging_fraction is set=0.6, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6
[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
[LightGBM] [Warning] bagging_fraction is set=0.6, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6
[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
[LightGBM] [Warning] bagging_fraction is set=0.6, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6
[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
[LightGBM] [Warning] bagging_fraction is set=0.6, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6
[LightGBM] [Warning] feature_fractio

In [50]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)
pipeline_light_2.fit(X_train, y_train)
y_val_pred_i = pipeline_light_2.predict(X_val)
medae_all = median_absolute_error(y_val, y_val_pred)
print('MedAE using all features: {}'.format(medae_all))

for i,k in enumerate(medae_dict):
    print('MedAE when removing {}: {}'.format(k, medae_dict[k]))


[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
[LightGBM] [Warning] bagging_fraction is set=0.6, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6
MedAE using all features: 0.6527384090608557
MedAE when removing allelectrons_Total: 0.5581649783897231
MedAE when removing density_Total: 0.5468895977000372
MedAE when removing allelectrons_Average: 0.5493471907771816
MedAE when removing val_e_Average: 0.5552067201290045
MedAE when removing atomicweight_Average: 0.5549171638250172
MedAE when removing ionenergy_Average: 0.5550724789371309
MedAE when removing el_neg_chi_Average: 0.5402329076079049
MedAE when removing R_vdw_element_Average: 0.5322745309485315
MedAE when removing R_cov_element_Average: 0.5367730990769446
MedAE when removing zaratio_Average: 0.5422356566340829
MedAE when removing 

Median Absolute Error less affected by removing:
> 1. total number of electrons
> 2. average number of valence electrons
> 3. average atomic weight

This is consistent with the correlation results.

The score improves when removing:
> 1. covalent or van der Walls radii
> 2. average density


In [58]:
X_corr_2 = X.drop(columns=['R_vdw_element_Average', 'density_Average'])

X_train_corr_2, X_val_corr_2, y_train_corr_2, y_val_corr_2 = train_test_split(X_corr_2, y, test_size = 0.3, random_state = 42)

pipeline_light_2.fit(X_train_corr_2, y_train_corr_2)
y_val_pred_corr_2 = pipeline_light_2.predict(X_val_corr_2)
medae_corr_2 = median_absolute_error(y_val_corr_2, y_val_pred_corr_2)
print(medae_corr_2)


[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
[LightGBM] [Warning] bagging_fraction is set=0.6, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6
0.522685313716571


Kaggle Score: 0.5153

In [56]:
# pipeline_light_2.fit(X_corr_2,y)

# X_test_corr_2 = X_test.drop(columns=['R_vdw_element_Average', 'density_Average'])
# y_test_pred = pipeline_light_2.predict(X_test_corr_2)

# output = pd.DataFrame({'id': X_test_corr_2.index,
#                        'emission': y_test_pred})

# from datetime import date, datetime
# now = datetime.now()
# today_str = now.strftime("%d%m%Y_%H%M")

# subm_file_name = "submissions/submission_"+today_str+".csv"

# output.to_csv(subm_file_name, index=False)

### Using RobustScaler

In [25]:
process = preprocessing.RobustScaler()

pipeline_light_3 = Pipeline(steps=[('processing',process),
                     ('regressor', lgb.LGBMRegressor(random_state=42, bagging_fraction = 0.6,
                                                          boosting = 'gbdt', feature_fraction = 0.6,
                                                          learning_rate = 0.05, max_depth = 15,
                                                          num_leaves = 15, objective= 'mae'))])

pipeline_rf_3 = Pipeline(steps = [('processing', process),
                                ('regressor',RandomForestRegressor(max_depth=15,random_state = 42))])

pipeline_dt_3 = Pipeline(steps=[('processing', process),
                              ('regressor', DecisionTreeRegressor(max_depth=15,random_state = 42))])

pipeline_svr_3 = Pipeline(steps=[('processing', process),
                              ('regressor', SVR(degree = 6, gamma = 'scale', kernel = 'poly'))])


pipelines_3 = [pipeline_base, pipeline_light_3, pipeline_rf_3, pipeline_dt_3, pipeline_svr_3]
pipe_dict_3 = {0: 'Baseline', 1: 'LightGBM', 2:'Random Forest', 3: 'Decision Tree', 4: 'SVR'}

for pipe in pipelines_3:
    pipe.fit(X_train_corr, y_train_corr)

best_precision = 0.0
best_regressor = 0
best_medae = 10
best_pipeline = ''
prediction_dict_3={}

for i, model in enumerate (pipelines_3) : 
    y_val_pred = model.predict(X_val_corr)
    medae = median_absolute_error(y_val_corr, y_val_pred)

    prediction_dict_3[pipe_dict_3[i]] = medae

    if medae < best_medae:
        best_medae = medae
        best_pipeline = model
        best_regressor = i
print('Best Median Absolute Error with {} | MedAE: {}'.format(pipe_dict_3[best_regressor],best_medae))

prediction_dict_3

[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
[LightGBM] [Warning] bagging_fraction is set=0.6, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6
Best Median Absolute Error with LightGBM | MedAE: 0.5878001035211016


{'Baseline': 0.9935564263469043,
 'LightGBM': 0.5878001035211016,
 'Random Forest': 0.670857332556424,
 'Decision Tree': 0.6348837209302376,
 'SVR': 0.9915724532192645}

In [57]:
pipeline_light_3.fit(X_train_corr_2, y_train_corr_2)
y_val_pred_corr_3 = pipeline_light_3.predict(X_val_corr_2)
medae_corr_2 = median_absolute_error(y_val_corr_2, y_val_pred_corr_3)
print(medae_corr_2)

[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
[LightGBM] [Warning] bagging_fraction is set=0.6, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6
0.5299773378723014


In [59]:
# pipeline_light_3.fit(X_corr_2,y)

# X_test_corr_3 = X_test.drop(columns=['R_vdw_element_Average', 'density_Average'])
# y_test_pred = pipeline_light_3.predict(X_test_corr_3)

# output = pd.DataFrame({'id': X_test_corr_3.index,
#                        'emission': y_test_pred})

# from datetime import date, datetime
# now = datetime.now()
# today_str = now.strftime("%d%m%Y_%H%M")

# subm_file_name = "submissions/submission_"+today_str+".csv"

# output.to_csv(subm_file_name, index=False)

[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
[LightGBM] [Warning] bagging_fraction is set=0.6, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6


Kaggle Score: 0.5130

In [28]:
print(prediction_dict)
print(prediction_dict_2)
print(prediction_dict_3)

{'Baseline': 0.9555663913173742, 'LightGBM': 0.6452822745961022, 'Random Forest': 0.645478039694277, 'Decision Tree': 0.7000000000000002, 'SVR': 0.6527384090608557}
{'Baseline': 0.9935564263469043, 'LightGBM': 0.5845266262717335, 'Random Forest': 0.6731373350042418, 'Decision Tree': 0.6348837209302376, 'SVR': 0.7117191991904059}
{'Baseline': 0.9935564263469043, 'LightGBM': 0.5878001035211016, 'Random Forest': 0.670857332556424, 'Decision Tree': 0.6348837209302376, 'SVR': 0.9915724532192645}


Notes from model optimization:
> 1. RobustScaler improves performance slightly
> 2. Reducing the number of features seems to improve MedAE
> 3. LightGBM seems, overall, the best model